In [5]:
# 기본 패키지
import re
import time
import pickle
import pandas as pd
from tqdm import tqdm
from tqdm import trange
import warnings
warnings.filterwarnings('ignore')

# 크롤링 패키지
import requests
from selenium import webdriver

# slack으로 작업 완료 메시지 전송을 위한 사용자 정의 로컬 패키지
import myslack

In [6]:
# 0. 페이지 한번 맨아래로 내리기
def scroll_bottom():
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")

# 1. 입력한 위치로 설정하기
def set_location(location):
    print(location+'으로 위치 설정 하는중...')
    driver.find_element_by_css_selector('#search > div > form > input').click()
    driver.find_element_by_css_selector('#button_search_address > button.btn-search-location-cancel.btn-search-location.btn.btn-default > span').click()
    driver.find_element_by_css_selector('#search > div > form > input').send_keys(location)
    driver.find_element_by_css_selector('#button_search_address > button.btn.btn-default.ico-pick').click()
    time.sleep(2)
    print(location+'으로 위치 설정 완료!')
    
# 2-1. 카테고리 페이지 Element Number Dictionary 정의
food_dict = { '프랜차이즈':3, '치킨':4, '피자&양식':5, '중국집':6,
              '한식':7, '일식&돈까스':8, '족발&보쌈':9,
              '야식':10, '분식':11, '카페&디저트':12 }

# 2-2. 카테고리 페이지로 넘어가기
def go_to_category(category):
    print(category+' 카테고리 페이지 로드중...')
    driver.find_element_by_xpath('//*[@id="category"]/ul/li[{}]/span'.format(food_dict.get(category))).click()
    time.sleep(4)
    print(category+' 카테고리 페이지 로드 완료!')    

# 3. 카테고리(음식점 리스트) 페이지 모두 펼치기
def stretch_list_page():
    restaurant_count = int(driver.find_element_by_css_selector('#restaurant_count').text)
    scroll_count = int((restaurant_count/20))
    print('모든 음식점 리스트 불러오기 시작...')
    for _ in trange(scroll_count):
        try:
            scroll_bottom()
            time.sleep(5)
        except Exception as e:
            pass
    scroll_bottom()
    time.sleep(5)
    print('모든 음식점 리스트 불러오기 완료!')
    
# 4. 해당 카테고리 음식점 리스트 리턴
def get_restaurant_list():
    restaurant_list=[]
    restaurants = driver.find_elements_by_css_selector('#content > div > div.restaurant-list > div.col-sm-6.contract')
    for restaurant in restaurants:
        restaurant_list.append(restaurant.find_element_by_css_selector('div > table > tbody > tr > td:nth-child(2) > div > div.restaurant-name.ng-binding').text)
    return list(set(restaurant_list))

# 5. 검색창에 음식점 검색하기
def search_restaurant(restaurant_name):
    try:
        driver.find_element_by_xpath('//*[@id="category"]/ul/li[1]/a').click()
        driver.find_element_by_xpath('//*[@id="category"]/ul/li[13]/form/div/input').send_keys(restaurant_name)    
        driver.find_element_by_xpath('//*[@id="category_search_button"]').click()
    except Exception as e:
        print('search_restaurant 에러')
    time.sleep(5)

# 6. 검색한 음식점 클릭
def go_to_restaurant():
    try:
        driver.find_element_by_xpath('//*[@id="content"]/div/div[4]/div[2]/div').click()
    except Exception as e:
        print('go_to_restaurant 에러')
    time.sleep(5)

# 7. 해당 음식점의 리뷰 페이지로 넘어가기
def go_to_review():
    print('리뷰 페이지 로드중...')
    driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[2]/a').click()
    time.sleep(2)
    print('리뷰 페이지 로드 완료!')
    
# 8. 리뷰 더보기 클릭하기 
def click_more_review():
    driver.find_element_by_class_name('btn-more').click()
    time.sleep(2)
    
# 9. 리뷰 페이지 모두 펼치기
def stretch_review_page():
    review_count = int(driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[2]/a/span').text)
    click_count = int((review_count/10))
    print('모든 리뷰 불러오기 시작...')
    for _ in trange(click_count):
        try:
            scroll_bottom()
            click_more_review()
        except Exception as e:
            pass
    scroll_bottom()
    print('모든 리뷰 불러오기 완료!')
        
# 10. 해당 음식점의 모든 리뷰 객체 리턴
def get_all_review_elements():
    reviews = driver.find_elements_by_css_selector('#review > li.list-group-item.star-point.ng-scope')
    return reviews
        
# 11. 페이지 뒤로 가기 (한 음식점 리뷰를 모두 모았으면 다시 음식점 리스트 페이지로 돌아감)
def go_back_page():
    print('페이지 돌아가기중...')
    driver.execute_script("window.history.go(-1)")
    time.sleep(5)
    print('페이지 돌아가기 완료!'+'\n')
    
# 12. 크롤링과 결과 데이터를 pickle 파일로 저장
def save_pickle(location, category, yogiyo_df):
    pickle.dump(yogiyo_df, open('./data/{}_{}_df.pkl'.format(location, category),'wb'))
    print('{} {} pikcle save complete!'.format(location, category))

In [7]:
# 13. 크롤링 메인 함수
def yogiyo_crawling(location, category):
    df = pd.DataFrame(columns=['Restaurant','UserID','Menu','Review',
                                   'Total','Taste','Quantity','Delivery','Date'])
    
    try:
        set_location(location) # 검색할 지역 설정
        go_to_category(category) # 해당 카테고리(음식점 리스트) 페이지로 넘어감

        print('Start [ {} - {} ] Crawling...'.format(location, category))
        
        stretch_list_page() # 카테고리(음식점 리스트) 페이지 모두 펼치기
        restaurant_list = get_restaurant_list() # 해당 카테고리 음식점 리스트 받아오기
        
        for restaurant_name in restaurant_list:
            try:
                print('********** '+restaurant_name+' ( '+str(restaurant_list.index(restaurant_name)+1)
                      +'/'+str(len(restaurant_list))+' 번째) **********')
                
                search_restaurant(restaurant_name) # 음식점을 순서대로 검색
                go_to_restaurant() # 검색한 음식점 클릭             
                go_to_review() # 해당 음식점의 리뷰페이지로 넘어감
                stretch_review_page() # 해당 음식점의 모든 리뷰를 불러옴

                for review in tqdm(get_all_review_elements()):  # 해당 음식점의 리뷰 수 만큼 데이터를 가져옴
                    try:
                        df.loc[len(df)] = { 
                            'Restaurant':driver.find_element_by_class_name('restaurant-name').text,
                            'UserID':review.find_element_by_css_selector('span.review-id.ng-binding').text,
                            'Menu':review.find_element_by_css_selector('div.order-items.default.ng-binding').text,
                            'Review':review.find_element_by_css_selector('p').text,
                            'Total':str(len(review.find_elements_by_css_selector('div > span.total > span.full.ng-scope'))),
                            'Taste':review.find_element_by_css_selector('div:nth-child(2) > div > span.category > span:nth-child(3)').text,
                            'Quantity':review.find_element_by_css_selector('div:nth-child(2) > div > span.category > span:nth-child(6)').text,
                            'Delivery':review.find_element_by_css_selector('div:nth-child(2) > div > span.category > span:nth-child(9)').text,
                            'Date':review.find_element_by_css_selector('div:nth-child(1) > span.review-time.ng-binding').text,
                        }
                    except Exception as e:
                        print('리뷰 페이지 에러')
                        print(e)
                        pass
                    
            except Exception as e:
                print('*** '+restaurant_name+' *** 음식점 페이지 에러')
                go_back_page()
                print(e)
                pass
            
            print('음식점 리스트 페이지로 돌아가는중...')
            go_back_page() # 해당 음식점 리뷰를 모두 모았으면 다시 음식점 리스트 페이지로 돌아감
            
    except Exception as e:
        print('음식점 리스트 페이지 에러')
        print(e)
        pass
        
    print('End of [ {} - {} ] Crawling!'.format(location, category))
    save_pickle(location, category, df) # 해당 음식점 리뷰 데이터를 pickle 파일로 저장함
    myslack.send_slack('{} {} crawling finish!!!'.format(location, category))
    
    return df 

In [37]:
# 14. 서초구 모든 관할구역(동) 크롤링 실행 함수
def start_seochogu_crawling(category):

    seochogu = ['반포동','내곡동','방배동','서초동','신원동','양재동','염곡동','우면동','원지동','잠원동']
    
    for dong in seochogu:
        try:
            yogiyo = yogiyo_crawling('서초구 {}'.format(dong), category)
            print(dong+' - '+category+', shape: '+str(yogiyo.shape)+'\n')
        except Exception as e:
            print('***** '+dong+' 에러 발생 *****')
            print(e)
            pass

---

In [14]:
# Chrome webdriver - 요기요 메인 페이지 실행
driver = webdriver.Chrome() 
url = 'https://www.yogiyo.co.kr/'
driver.get(url)

In [38]:
%time start_seochogu_crawling('치킨')

서초구 우면동으로 위치 설정 하는중...
서초구 우면동으로 위치 설정 완료!
치킨 카테고리 페이지 로드중...


0it [00:00, ?it/s]

치킨 카테고리 페이지 로드 완료!
Start [ 서초구 우면동 - 치킨 ] Crawling...
모든 음식점 리스트 불러오기 시작...


모든 음식점 리스트 불러오기 완료!
********** 처갓집양념치킨-양재점 ( 1/15 번째) **********
리뷰 페이지 로드중...


  0%|          | 0/36 [00:00<?, ?it/s]

...

리뷰 페이지 로드 완료!
모든 리뷰 불러오기 시작...


  0%|          | 0/180 [00:00<?, ?it/s]

모든 리뷰 불러오기 완료!


100%|██████████| 180/180 [00:53<00:00,  3.39it/s]


음식점 리스트 페이지로 돌아가는중...
페이지 돌아가기중...
페이지 돌아가기 완료!

End of [ 서초구 우면동 - 치킨 ] Crawling!
서초구 우면동 치킨 pikcle save complete!
<Response [200]>
우면동 - 치킨, shape: (2969, 9)

CPU times: user 37.3 s, sys: 2.75 s, total: 40 s
Wall time: 1h 11min 55s


In [39]:
df = pickle.load(open('./data/서초구 우면동_치킨_df.pkl','rb'))

In [40]:
df.tail()

,Restaurant,UserID,Menu,Review,Total,Taste,Quantity,Delivery,Date
2964,60계-양재점,no**님,"간지치킨＋고추치킨/1(음료 사이즈 변경(콜라1.25L로 변경),추가선택(양념소스 추가))",일단 배달해주시는분께서 친절하셔서 기분좋게 먹었네요~~ 처음에 봤을땐 양이적어보였는...,5,5,5,5,2018년 2월 27일
2965,60계-양재점,dp**님,"후라이드＋고추치킨/1(음료 사이즈 변경(콜라1.25L로 변경),추가선택(마요네즈소스...",또 시켜먹을게요~잘먹었습니다,4,4,4,4,2018년 2월 27일
2966,60계-양재점,ox**님,양념치킨-순살/1,맛있게먹었습니다~~,5,5,5,5,2018년 2월 26일
2967,60계-양재점,li**님,간지치킨＋고추치킨/1,새로 먹어본 메뉴인데 괜찮네요.,5,5,5,5,2018년 2월 26일
2968,60계-양재점,su**님,（추천）후라이드/1,60계 생겨서 너무좋아요~ 맛있어요~!!,5,5,5,5,2018년 2월 22일


In [41]:
df['Restaurant'].value_counts()

BHC-우면점          785
BHC-양재점          727
또래오래-양재1동우면점     436
처갓집양념치킨-양재점      365
60계-양재점          180
꿍닭꿍닭-본점           90
호식이두마리치킨-양재점      90
강정이기가막혀-양재우면점     78
추억의통치킨            48
치킨마루-양재언남점        43
가마로강정-양재역점        42
페리카나-우면동점         39
네네치킨-서초우면점        37
JS치코파닭             7
통치킨앤피자             2
Name: Restaurant, dtype: int64

In [42]:
# Chrome webdriver 종료
driver.close() 
driver.quit() 

---